This is a tutorial to lean how LDA works under the hood, based on the tutorial by andrew woods, http://brooksandrew.github.io/simpleblog/articles/latent-dirichlet-allocation-under-the-hood/

In [8]:
import numpy as np
import operator
from functools import reduce

Generate a corpus

In [9]:
rawdocs = ['eat turkey on turkey day holiday',
          'i like to eat cake on holiday',
          'turkey trot race on thanksgiving holiday',
          'snail race the turtle',
          'time travel space race',
          'movie on thanksgiving',
          'movie at air and space museum is cool movie',
          'aspiring movie star']

Convert raw docs into bag of words docs

In [10]:
docs_bow = [ d.split(' ') for d in rawdocs]
print(docs_bow)

[['eat', 'turkey', 'on', 'turkey', 'day', 'holiday'], ['i', 'like', 'to', 'eat', 'cake', 'on', 'holiday'], ['turkey', 'trot', 'race', 'on', 'thanksgiving', 'holiday'], ['snail', 'race', 'the', 'turtle'], ['time', 'travel', 'space', 'race'], ['movie', 'on', 'thanksgiving'], ['movie', 'at', 'air', 'and', 'space', 'museum', 'is', 'cool', 'movie'], ['aspiring', 'movie', 'star']]


Construct vocabulary by obtaining unique words from the document

In [ ]:

vocabulary = np.unique( reduce( operator.concat, docs_bow) ).tolist()
print(vocabulary)

['air', 'and', 'aspiring', 'at', 'cake', 'cool', 'day', 'eat', 'holiday', 'i', 'is', 'like', 'movie', 'museum', 'on', 'race', 'snail', 'space', 'star', 'thanksgiving', 'the', 'time', 'to', 'travel', 'trot', 'turkey', 'turtle']


Replace the words in documents with the index from the vocabulary

In [14]:
docs = []
for d in  docs_bow:
    docs.append([vocabulary.index(w) for w in d])
print(docs)

[[7, 25, 14, 25, 6, 8], [9, 11, 22, 7, 4, 14, 8], [25, 24, 15, 14, 19, 8], [16, 15, 20, 26], [21, 23, 17, 15], [12, 14, 19], [12, 3, 0, 1, 17, 13, 10, 5, 12], [2, 12, 18]]
